# Libbitcoin BX: Transactions with relative locktime.
In this BX tutorial, we will send funds to a `checksequenceverify` encumbered output, and spend this after the locktime delay (which begins when the uxto is included in a block) has expired.
<img src="images/csv_overview.jpg" alt="drawing" style="" width="700px"/>

<hr style="border: 0.5px dashed #000;">

## 1) Setup wallet and test funds
### 1.1) Generate private/public key pair
#### 1.1.1) New private key

In [ ]:
bx seed | bx ec-new

#### 1.1.2) Save your private key

In [ ]:
echo 962f3488480b23d4f85a198ba11e862ee6eebf4259af3f153a7c20084573b47c

#### 1.1.3) Derive public key<br>

In [ ]:
bx ec-to-public 962f3488480b23d4f85a198ba11e862ee6eebf4259af3f153a7c20084573b47c

#### 1.1.4) Derive public key hash

In [ ]:
bx sha256 035d2725b8a5b78f8e235a92e3acc2e426ac70889785dba7020189e88caa8a45b6 | bx ripemd160

### 1.2) Derive testnet address
Add version prefix for mainnet/testnet<br>
* `6f` in hex
* `111` in decimal

And base58 encode result

In [ ]:
bx wrap-encode -v 111 903d20a04087a7a843f1f6fe0dfc8ae7358e058d \
| bx base58-encode


### 1.3) Send test coins funds to your testnet address
Note your spendable p2pkh UXTO's below...

* UXTO 0: 
    * TXID: `5509739eb377fc44a0c5b7a5765ab7c759f59914af76b85561671feda313af8b`<br>
    * Index: `0`
    * Amount: `129998350` 
    
* UXTO 1: (spent)
    * TXID: `6e470b4c9ac631934ebb0f5e58b3c32a6a7278840b7ec9fe0cceac1dc3f9c29d`<br>
    * Index: `0`
    * Amount: `130000000`
     
    

<hr style="border: 0.5px dashed #000;">

## 2) Spending to a CSV encumbered output<br>
<img src="images/spend_to_csv.jpg" alt="drawing" style="" width="700px"/>

### 2.1) Build transaction with CSV encumbered output for signing

#### 2.1.1) Determine delay in blocks
* Encode delay value in hex (little endian)
* `CHECKSEQUENCEVERIFY` verifies a 3-byte `[delay]` value on the stack against the spending transaction<br>
* For a `[delay]` in blocks:
    * Bit flag (1 << 22) is not set.
    * First 16 bits interpreted as blocks.
    * Little endian encoded.  

In [ ]:
# A five block delay: 0x000005


**Note:** Don't forget to flip byte-order (3-Bytes) above for output script below.

#### 2.1.2) Encode CSV-encumbered P2PKH output script
* **`[delay]`**
* **`CHECKSEQUENCEVERIFY`**
* `DROP`
* `DUP`
* `HASH160`
* `[public key hash]`
* `EQUALVERIFY`
* `CHECKSIG`

In [ ]:
bx script-encode \
"[040000] CHECKSEQUENCEVERIFY DROP DUP HASH160 [903d20a04087a7a843f1f6fe0dfc8ae7358e058d] EQUALVERIFY CHECKSIG"


#### 2.1.3) Encode output amount
* `400 Satoshi/sigops + 1 Satoshi/Byte` 
* Output amount(s) are previous output amount(s) minus fees

In [ ]:
expr 130000000 - 4 \* 100 - 250 - 200

#### 2.1.4) Encode transaction for signing
* `bx tx-encode -i [prevout txid:index:sequence] -o [output script:amount]`


In [ ]:
bx tx-encode \
--input 6e470b4c9ac631934ebb0f5e58b3c32a6a7278840b7ec9fe0cceac1dc3f9c29d:0 \
--output 03040000b27576a914903d20a04087a7a843f1f6fe0dfc8ae7358e058d88ac:129999150


### 2.2) Building the input script

#### 2.2.1) Signing the transaction with the private key
* `bx fetch-tx` previous output script
* `bx input-sign` `[private key]` `["previous output script"]` `[encoded transaction]`
    * Outputs DER encoded signature

In [ ]:
bx fetch-tx -f json 6e470b4c9ac631934ebb0f5e58b3c32a6a7278840b7ec9fe0cceac1dc3f9c29d \
| jq ".transaction.outputs[0].script"

In [ ]:
bx input-sign \
962f3488480b23d4f85a198ba11e862ee6eebf4259af3f153a7c20084573b47c \
"dup hash160 [903d20a04087a7a843f1f6fe0dfc8ae7358e058d] equalverify checksig" \
01000000019dc2f9c31dacce0cfec97e0b8478726a2ac3b3585e0fbb4e9331c69a4c0b476e0000000000ffffffff012ea1bf07000000001f03040000b27576a914903d20a04087a7a843f1f6fe0dfc8ae7358e058d88ac00000000


#### 2.2.2) Set the input script into the finalised transaction
* The input script to spend the previous p2pkh output script is `[endorsement]` `[public key]`
* `bx input-set` `"input script"` `[encoded transaction]`

In [ ]:
bx input-set \
"[3045022100e9d38f238e58b3eb4642a618e0478bde90e33e5f17749336c97d8057f1b0a3d6022070e7c2f2b9cf4f0d771889dd6488d399889408ff4985ea2def6ede808de69b7601] [035d2725b8a5b78f8e235a92e3acc2e426ac70889785dba7020189e88caa8a45b6]" \
01000000019dc2f9c31dacce0cfec97e0b8478726a2ac3b3585e0fbb4e9331c69a4c0b476e0000000000ffffffff012ea1bf07000000001f03040000b27576a914903d20a04087a7a843f1f6fe0dfc8ae7358e058d88ac00000000


### 2.3) Validate & Broadcast the endorsed transaction
* `bx validate-tx` `[encoded transaction]`
* `bx tx-decode -f json` `[encoded transaction]` | `jq ".transaction.hash"`

In [ ]:
bx validate-tx \
01000000019dc2f9c31dacce0cfec97e0b8478726a2ac3b3585e0fbb4e9331c69a4c0b476e000000006b483045022100e9d38f238e58b3eb4642a618e0478bde90e33e5f17749336c97d8057f1b0a3d6022070e7c2f2b9cf4f0d771889dd6488d399889408ff4985ea2def6ede808de69b760121035d2725b8a5b78f8e235a92e3acc2e426ac70889785dba7020189e88caa8a45b6ffffffff012ea1bf07000000001f03040000b27576a914903d20a04087a7a843f1f6fe0dfc8ae7358e058d88ac00000000


In [ ]:
bx tx-decode -f json \
01000000019dc2f9c31dacce0cfec97e0b8478726a2ac3b3585e0fbb4e9331c69a4c0b476e000000006b483045022100e9d38f238e58b3eb4642a618e0478bde90e33e5f17749336c97d8057f1b0a3d6022070e7c2f2b9cf4f0d771889dd6488d399889408ff4985ea2def6ede808de69b760121035d2725b8a5b78f8e235a92e3acc2e426ac70889785dba7020189e88caa8a45b6ffffffff012ea1bf07000000001f03040000b27576a914903d20a04087a7a843f1f6fe0dfc8ae7358e058d88ac00000000 \
|jq ".transaction.hash"


In [ ]:
bx send-tx 01000000019dc2f9c31dacce0cfec97e0b8478726a2ac3b3585e0fbb4e9331c69a4c0b476e000000006b483045022100e9d38f238e58b3eb4642a618e0478bde90e33e5f17749336c97d8057f1b0a3d6022070e7c2f2b9cf4f0d771889dd6488d399889408ff4985ea2def6ede808de69b760121035d2725b8a5b78f8e235a92e3acc2e426ac70889785dba7020189e88caa8a45b6ffffffff012ea1bf07000000001f03040000b27576a914903d20a04087a7a843f1f6fe0dfc8ae7358e058d88ac00000000 


<hr style="border: 0.5px dashed #000;">

## 3) Spend the CSV encumbered UXTO

<img src="images/spend_from_csv.jpg" alt="drawing" style="" width="700px"/>

### 3.1) Build transaction with activated delay locktime

#### 3.1.1) Encode p2pkh output script

In [ ]:
bx script-encode "DUP HASH160 [903d20a04087a7a843f1f6fe0dfc8ae7358e058d] EQUALVERIFY CHECKSIG"

**3.1.2) Encode transaction with delay & set input sequence**<br>
* Transaction sequence delay must be `>=` delay in CSV output being spent.
* Sequence delay encoding in blocks:
    * Transaction version must be >= 2.
    * Bit flag (1 << 22) must be deactivated.
    * Encode block delay in first 16 bits.

In [ ]:
expr 129999350 - 4 \* 100 - 300 - 300

In [ ]:
bx tx-encode \
--version 2 \
--input 16f7fbce888a8001d99f92519eff7d4755fee947a0b9289ced5c5f723d3671e0:0:5 \
--output 76a914903d20a04087a7a843f1f6fe0dfc8ae7358e058d88ac:129998350


### 3.2) Building the input script

#### 3.2.1)  Signing the transaction with the private key
* `bx fetch-tx` previous output script
* `bx input-sign` `[private key]` `["previous output script"]` `[encoded transaction]`
    * Outputs DER encoded signature

In [ ]:
bx fetch-tx -f json 16f7fbce888a8001d99f92519eff7d4755fee947a0b9289ced5c5f723d3671e0 \
| jq ".transaction.outputs[0].script"


In [ ]:
bx input-sign 962f3488480b23d4f85a198ba11e862ee6eebf4259af3f153a7c20084573b47c \
"[050000] checksequenceverify drop dup hash160 [42a8bcd5f61fa17adbe800de6382f24bd1db0d7f] equalverify checksig" \
0200000001e071363d725f5ced9c28b9a047e9fe55477dff9e51929fd901808a88cefbf716000000000005000000010e9ebf07000000001976a914903d20a04087a7a843f1f6fe0dfc8ae7358e058d88ac00000000


#### 3.2.2) Set the input script into the finalised transaction

In [ ]:
bx input-set "[3045022100d6a7eba49a4bfecccc5519a9ed25b0de32c3dc487c37b9b36b85ea9526bede47022052dd8005ff085ae56d10e84a173fd2128d90f682b5abc8d60f6db1f50526a93801] [035d2725b8a5b78f8e235a92e3acc2e426ac70889785dba7020189e88caa8a45b6]" \
0200000001e071363d725f5ced9c28b9a047e9fe55477dff9e51929fd901808a88cefbf716000000000005000000010e9ebf07000000001976a914903d20a04087a7a843f1f6fe0dfc8ae7358e058d88ac00000000


### 3.3) Validate & Broadcast the endorsed transaction
* `bx validate-tx` `[encoded transaction]`
* Get TXID: `bx tx-decode -f json` `[encoded transaction]` | `jq ".transaction.hash"`
* `bx send-tx` `[encoded transaction]`

In [ ]:
bx validate-tx \
0200000001e071363d725f5ced9c28b9a047e9fe55477dff9e51929fd901808a88cefbf716000000006b483045022100d6a7eba49a4bfecccc5519a9ed25b0de32c3dc487c37b9b36b85ea9526bede47022052dd8005ff085ae56d10e84a173fd2128d90f682b5abc8d60f6db1f50526a938012102e1af2f9086e95728bc3c768ee3e83ab81dd1841b92e281caa9f92501fd9b2b3105000000010e9ebf07000000001976a914903d20a04087a7a843f1f6fe0dfc8ae7358e058d88ac00000000


In [ ]:
bx tx-decode -f json \
0200000001e071363d725f5ced9c28b9a047e9fe55477dff9e51929fd901808a88cefbf716000000006b483045022100d6a7eba49a4bfecccc5519a9ed25b0de32c3dc487c37b9b36b85ea9526bede47022052dd8005ff085ae56d10e84a173fd2128d90f682b5abc8d60f6db1f50526a938012102e1af2f9086e95728bc3c768ee3e83ab81dd1841b92e281caa9f92501fd9b2b3105000000010e9ebf07000000001976a914903d20a04087a7a843f1f6fe0dfc8ae7358e058d88ac00000000 \
| jq ".transaction.hash"


In [ ]:
bx send-tx 0200000001e071363d725f5ced9c28b9a047e9fe55477dff9e51929fd901808a88cefbf716000000006b483045022100d6a7eba49a4bfecccc5519a9ed25b0de32c3dc487c37b9b36b85ea9526bede47022052dd8005ff085ae56d10e84a173fd2128d90f682b5abc8d60f6db1f50526a938012102e1af2f9086e95728bc3c768ee3e83ab81dd1841b92e281caa9f92501fd9b2b3105000000010e9ebf07000000001976a914903d20a04087a7a843f1f6fe0dfc8ae7358e058d88ac00000000
